# Downloading data using netCDF Subset Service (NCSS)


This notebook shows ways to list all the data collection, sub-collections, datasets and files from NCI's THREDDS service portal. 

* NetCDF Subset Service (NCSS)
* Use Siphon to make an NCSS request
* Plot requested data using CartoPy
* NCSS point request 
* Construct NCSS query through the THREDDS NCSS service end point

---
This notebook is licenced under the [Creative Commons Attribution 4.0 International license](https://creativecommons.org/licenses/by/4.0/)

### THREDDS Netcdf Subset Service (NCSS)

The Netcdf Subset Service (NCSS) is one of the data service endpoints available on many of NCI's hosted data collections. NCSS is currently available on NCI's gridded datasets and allows users to request spatial and temporal queries using longitude and latitude values directly.

 The cell below uses Siphon to extract the THREDDS Data Server information of interest:

In [ ]:
from siphon.catalog import TDSCatalog

cat = TDSCatalog("http://thredds.nci.org.au/thredds/catalog/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r11i1p1f1/day/pr/gn/latest/catalog.xml")

ncss=cat.datasets[-1].subset()
cat.datasets[-1]

In [ ]:
ncss.variables

We can build a query to ask for the data we want from the THREDDS server:

In [ ]:
from datetime import date, timedelta

# Create a new NCSS query
query = ncss.query()

# Request data in netCDF format
query.accept('netcdf')

# Ask for our variable of interest
query.variables('pr')

# Set the time range of the data we want
start = date.fromisoformat('2013-12-31')
query.time_range(start, start + timedelta(days=365))

# Set the spatial limits - in this case, Australia
query.lonlat_box(west=110, east=160, north=-10, south=-50)

# get the data!
data = ncss.get_data(query)

Store the netCDF data:

In [ ]:
from xarray.backends import NetCDF4DataStore
import xarray as xr
import os

### create output directory
outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)

ds = xr.open_dataset(NetCDF4DataStore(data))
ds.to_netcdf('./output/NCSS_reqest_data.nc')

### Visualisa the data

In [ ]:
var = ds.pr
var

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np

latitude = var.lat
longitude = var.lon
time_index =0

data = var[time_index].squeeze()

# Mask invalid data points
masked_data = np.ma.masked_invalid(data)

# Create meshgrid for coordinates if necessary
lon2d, lat2d = np.meshgrid(longitude, latitude)

# Check for NaNs in coordinate arrays
lon2d = np.ma.masked_invalid(lon2d)
lat2d = np.ma.masked_invalid(lat2d)

# Plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertCylindrical())

ax.pcolormesh(lon2d, lat2d, masked_data,
              transform=ccrs.PlateCarree(), zorder=0)
ax.set_title(str(var.time[time_index].values))

# Add geographic features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES, edgecolor='black')
ax.add_feature(cfeature.BORDERS)

ax.gridlines()

plt.show()

### NCSS Point request

We can also request data for a specfic lon/lat point, across vertical coordinates or times.

In [ ]:
point_query = ncss.query()
startd = date.fromisoformat('2013-12-31')
point_query.time_range(startd, startd + timedelta(days=365))
point_query.accept('netcdf')
point_query.variables('pr')
point_query.lonlat_point(149.13, -35.28)

# get the data! 
point_data = ncss.get_data(point_query)
ds = xr.open_dataset(NetCDF4DataStore(point_data))
ds.time


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
t = ds.time
p = ds.pr

fig, ax = plt.subplots()
ax.plot(t, p)

ax.set(xlabel='date', ylabel='precipitation',
       title='Precipitaion in 2014 at Canberra')
ax.grid()

plt.show()

### NCSS request through the THREDDS weblink

Begin by navigating to the THREDDS catalog page for the selected data file (https://thredds.nci.org.au/thredds/catalog/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-0/rcp85/mon/atmos/Amon/r1i1p1/latest/tas/catalog.html?dataset=rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-0/rcp85/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-0_rcp85_r1i1p1_200601-210012.nc):

![](images/ncss1_cmip5.png)

Next, decide whether you want to extract a spatial subset (right/pink arrow), defined by a bounding box, or a single point in space (left/green arrow), defined by a single latitude and longitude:

![](images/ncss2_cmip5.png)

Then proceed by selecting the desired variables, spatial (or lat/lon point) and temporal extents. As selections are made, the NCSS Request URL (near bottom of the page) will update with these parameters:

![](images/ncss3_cmip5.png)

Click 'Submit' and the request file should begin to download. Alternatively, the NCSS URL can be copied/pasted into a new web browser or used in workflows along with `wget` or `curl` commands.


### Summery

We demonstrate how to use Siphon to query the subset of a dataset and how to use THREDDS NCSS service to subset data. 